In [ ]:
import numpy as np
import pandas as pd
from sheets import download_ledger, load_ledger
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

In [ ]:
df = download_ledger()
df.head()

In [ ]:
df.shape

In [ ]:
owner = "DC"
df = df[df['Owner'].apply(lambda x: x.startswith(owner))]
df = df[df['Currency'] == 'USD']
df = df[df['TradeTime'] < datetime(2024, 4, 1)]


In [ ]:
df.shape

In [ ]:
pivot = df.pivot_table(index=['Platform', 'Symbol'], values='Amount', aggfunc='sum')
pivot = pivot.reset_index()
pivot[pivot['Amount'] > 0]

In [ ]:
mtch = df[df['Symbol'] == 'MTCH']
mtch

In [ ]:
# check if there are any transfer in
any(mtch['B/S'] == 'TransferIn')

In [ ]:
brokers = {broker:0 for broker in mtch['Platform'].unique()}
brokers

In [ ]:
for i, row in mtch.iterrows():
    if row['B/S'] == 'Bought':
        brokers[row['Platform']] += row['Amount'] * row['Price in INR']
    elif row['B/S'] == 'TransferOut':
        brokers['transfer'] = brokers[row['Platform']]
        brokers[row['Platform']] = 0
    elif row['B/S'] == 'TransferIn':
        brokers[row['Platform']] = brokers['transfer']
        brokers['transfer'] = 0
brokers

In [ ]:
pivot = mtch.pivot_table(index='Platform', values='Amount', aggfunc='sum').reset_index()
pivot

In [ ]:
pivot.loc[:,'Cost'] = pivot['Platform'].apply(lambda x: brokers[x])
pivot

In [ ]:
def compute_cost(df, debug=None):

    df.sort_values(['TradeTime', 'B/S'], ascending=[True, False], inplace=True)

    if debug in ['DEBUG', 'INFO']:
        display(df)

    brokers = {broker: {'q':0,'i': 0} for broker in df['Platform'].unique()}

    for _, row in df.iterrows():
        action = row['B/S'].lower()
        platform = row['Platform']

        if debug == 'INFO':
            display(row)
            display("Before")
            display(brokers)

        if action == 'bought':
            brokers[platform]['i'] += row['Amount'] * row['Price in INR']
            brokers[platform]['q'] += row['Amount']
        elif action == 'sold':
            brokers[platform]['i'] -= brokers[platform]['i'] / brokers[platform]['q'] * abs(row['Amount'])
            brokers[platform]['q'] -= abs(row['Amount'])
        elif action == 'transferout':
            brokers['transfer'] = brokers[platform]
            brokers[platform] = {'q':0,'i': 0}
        elif action == 'transferin':
            brokers[platform] = brokers['transfer']
            brokers['transfer'] = {'q':0,'i': 0}

        if debug == 'INFO':
            display("After")
            display(brokers)
    # brokers

    pivot = df.pivot_table(index='Platform', values='Amount', aggfunc='sum').reset_index()
    pivot.loc[:,'Cost'] = pivot['Platform'].apply(lambda x: brokers[x]['i'])

    pivot = pivot[pivot['Amount'] > 0]
    
    if debug in ['DEBUG', 'INFO']:
        display(pivot)
    return pivot

In [ ]:
compute_cost(mtch)

In [ ]:
stocks = ['NVDA']
for stock in stocks:
    # print(stock)
    stockdf = df[df['Symbol'] == stock]
    compute_cost(stockdf, debug='DEBUG')